In [1]:
from collections import defaultdict, OrderedDict

import numpy as np
import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchvision.datasets import *
from torchvision.transforms import *
import torch.optim as optim
import re
from torchsummary import summary

import torch_pruning as tp

from collections import defaultdict, OrderedDict

import numpy as np
import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchvision.datasets import *
from torchvision.transforms import *
import torch.optim as optim

In [2]:
class VGG(nn.Module):
    ARCH = [64, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

    def __init__(self) -> None:
        super().__init__()

        layers = []
        counts = defaultdict(int)

        def add(name: str, layer: nn.Module) -> None:
            layers.append((f"{name}{counts[name]}", layer))
            counts[name] += 1

        in_channels = 3
        for x in self.ARCH:
            if x != 'M':
                # conv-bn-relu
                add("conv", nn.Conv2d(in_channels, x, 3, padding=1, bias=False))
                add("bn", nn.BatchNorm2d(x))
                add("relu", nn.ReLU(True))
                in_channels = x
            else:
                # maxpool
                add("pool", nn.MaxPool2d(2))

        self.backbone = nn.Sequential(OrderedDict(layers))
        self.classifier = nn.Linear(512, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:

        # backbone: [N, 3, 32, 32] => [N, 512, 2, 2]
        x = self.backbone(x)

        # avgpool: [N, 512, 2, 2] => [N, 512]
        x = x.mean([2, 3])

        # classifier: [N, 512] => [N, 10]
        x = self.classifier(x)
        return x

vgg = VGG()
mobilenet_v3 = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v3_small', pretrained=True)



Using cache found in /Users/sathya/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/sathya/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sathya/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
image_size = 32
transforms = {
    "train": Compose([
        RandomCrop(image_size, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
    ]),
    "test": ToTensor(),
}
dataset = {}
for split in ["train", "test"]:
  dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transforms[split],
  )
dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=512,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
  )
test_input = next(iter(dataloader['test']))[0][:2,:]

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# summary(model,(3,32,32))

In [5]:
# Final Pruning
# sparsity_dict = {model.backbone.conv0: 0.15000000000000002, model.backbone.conv1: 0.15, model.backbone.conv2: 0.15, model.backbone.conv3: 0.15000000000000002, model.backbone.conv4: 0.20000000000000004, model.backbone.conv5: 0.20000000000000004, model.backbone.conv6: 0.45000000000000007}
# imp = tp.importance.MagnitudeImportance(p=1, group_reduction='mean')

# pruner = tp.pruner.MetaPruner( 
#     model,
#     test_input,
#     importance=imp,
#     pruning_ratio_dict = sparsity_dict, 
# #     ignored_layers=ignored_layers,
# )

# pruner.step()

In [6]:
# model.zero_grad()
# summary(model,(3,32,32))

In [7]:
def reformat_layer_name(str_data):

    split_data = str_data.split('.')

    for ind in range(len(split_data)):
        data = split_data[ind]
        if (data.isdigit()):
            split_data[ind] = "[" + data + "]"
    final_string = '.'.join(split_data)

    iters_a = re.finditer(r'[a-zA-Z]\.\[', final_string)
    indices = [m.start(0) + 1 for m in iters_a]
    iters = re.finditer(r'\]\.\[', final_string)
    indices.extend([m.start(0) + 1 for m in iters])

    final_string = list(final_string)
    final_string = [final_string[i] for i in range(len(final_string)) if i not in indices]

    str_data = ''.join(final_string)

    

    return str_data

import random

def generate_random_array(n):
    random_array = []
    
    # Generate a random number of values between 0 and n
    num_values = random.randint(0, n)
    
    # Generate the random array with unique values
    row = random.sample(range(0, n+1), num_values)
    random_array.append(row)
    
    return random_array[0]



In [8]:
fusing_layers = [
    'Conv2d',
    'BatchNorm2d',
    'ReLU',
    'Linear',
    'BatchNorm1d',
]


def summary_string_fixed(model, all_layers, input_size, model_name=None, batch_size=-1, dtypes=None):
    if dtypes is None:
        dtypes = [torch.FloatTensor] * len(input_size)
        
    def name_fixer(names):

        return_list = []
        for string in names:
            matches = re.finditer(r'\.\[(\d+)\]', string)
            pop_list = [m.start(0) for m in matches]
            pop_list.sort(reverse=True)
            if len(pop_list) > 0:
                string = list(string)
                for pop_id in pop_list:
                    string.pop(pop_id)
                string = ''.join(string)
            return_list.append(string)
        return return_list

    def register_hook(module, module_idx):
        def hook(module, input, output):
            nonlocal module_idx
            m_key = all_layers[module_idx][0]
            m_key = model_name + "." + m_key

            try:
                eval(m_key)
            except:
                m_key = name_fixer([m_key])[0]

            summary[m_key] = OrderedDict()
            summary[m_key]["type"] = str(type(module)).split('.')[-1][:-2]
            summary[m_key]["input_shape"] = list(input[0].size())
            summary[m_key]["input_shape"][0] = batch_size

            if isinstance(output, (list, tuple)):
                summary[m_key]["output_shape"] = [
                    [-1] + list(o.size())[1:] for o in output
                ]
            else:
                summary[m_key]["output_shape"] = list(output.size())
                summary[m_key]["output_shape"][0] = batch_size

            params = 0
            if hasattr(module, "weight") and hasattr(module.weight, "size"):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                summary[m_key]["trainable"] = module.weight.requires_grad
                summary[m_key]["weight_shape"] = module.weight.shape
            if hasattr(module, "bias") and hasattr(module.bias, "size"):
                params += torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]["nb_params"] = params

        if (
                not isinstance(module, nn.Sequential)
                and not isinstance(module, nn.ModuleList)
        ):
            hooks.append(module.register_forward_hook(hook))

    if isinstance(input_size, tuple):
        input_size = [input_size]

    x = [torch.rand(2, *in_size).type(dtype)
         for in_size, dtype in zip(input_size, dtypes)]

    summary = OrderedDict()
    hooks = []

    for module_idx, (layer_name, module) in enumerate(all_layers):
        register_hook(module, module_idx)

    model(*x)

    for h in hooks:
        h.remove()

    return summary

def layer_mapping(model):
    
    def get_all_layers(model, parent_name=''):
        layers = []

        def reformat_layer_name(str_data):
            try:
                split_data = str_data.split('.')
                for ind in range(len(split_data)):
                    data = split_data[ind]
                    if (data.isdigit()):
                        split_data[ind] = "[" + data + "]"
                final_string = '.'.join(split_data)

                iters_a = re.finditer(r'[a-zA-Z]\.\[', final_string)
                indices = [m.start(0) + 1 for m in iters_a]
                iters = re.finditer(r'\]\.\[', final_string)
                indices.extend([m.start(0) + 1 for m in iters])

                final_string = list(final_string)
                final_string = [final_string[i] for i in range(len(final_string)) if i not in indices]

                str_data = ''.join(final_string)

            except:
                pass

            return str_data

        for name, module in model.named_children():
            full_name = f"{parent_name}.{name}" if parent_name else name
            test_name = "model." + full_name
            try:
                eval(test_name)
                layers.append((full_name, module))
            except:
                layers.append((reformat_layer_name(full_name), module))
            if isinstance(module, nn.Module):
                layers.extend(get_all_layers(module, parent_name=full_name))
        return layers
    all_layers = get_all_layers(model)
    model_summary = summary_string_fixed(model, all_layers, (3, 64, 64), model_name='model')  # , device="cuda")

    name_type_shape = []
    for key in model_summary.keys():
        data = model_summary[key]
        if ("weight_shape" in data.keys()):
            name_type_shape.append([key, data['type'], data['weight_shape'][0]])
        #     else:
    #         name_type_shape.append([key, data['type'], 0 ])
    name_type_shape = np.asarray(name_type_shape)

    name_list = name_type_shape[:, 0]

    r_name_list = np.asarray(name_list)
    random_picks = np.random.randint(0, len(r_name_list), 10)
    test_name_list = r_name_list[random_picks]
    eval_hit = False
    for layer in test_name_list:
        try:
            eval(layer)

        except:
            eval_hit = True
            break
    if (eval_hit):
        fixed_name_list = name_fixer(r_name_list)
        name_type_shape[:, 0] = fixed_name_list

    layer_types = name_type_shape[:, 1]
    layer_shapes = name_type_shape[:, 2]
    mapped_layers = {'model_layer': [], 'Conv2d_BatchNorm2d_ReLU': [], 'Conv2d_BatchNorm2d': [], 'Linear_ReLU': [],
                     'Linear_BatchNorm1d': []}

    def detect_sequences(lst):
        i = 0
        while i < len(lst):

            if i + 2 < len(lst) and [l for l in lst[i: i + 3]] == [
                fusing_layers[0],
                fusing_layers[1],
                fusing_layers[2],
            ]:
                test_layer = layer_shapes[i: i + 2]
                if (np.all(test_layer == test_layer[0])):
                    mapped_layers['Conv2d_BatchNorm2d_ReLU'].append(
                        np.take(name_list, [i for i in range(i, i + 3)]).tolist()
                    )
                    i += 3

            elif i + 1 < len(lst) and [l for l in lst[i: i + 2]] == [
                fusing_layers[0],
                fusing_layers[1],
            ]:
                test_layer = layer_shapes[i: i + 2]
                if (np.all(test_layer == test_layer[0])):
                    mapped_layers['Conv2d_BatchNorm2d'].append(
                        np.take(name_list, [i for i in range(i, i + 2)]).tolist()
                    )
                    i += 2
            # if i + 1 < len(lst) and [ type(l) for l in lst[i:i+2]] == [fusing_layers[0], fusing_layers[2]]:
            #     detected_sequences.append(np.take(name_list,[i for i in range(i,i+2)]).tolist())
            #     i += 2
            # elif i + 1 < len(lst) and [ type(l) for l in lst[i:i+2]] == [fusing_layers[1], fusing_layers[2]]:
            #     detected_sequences.append(np.take(name_list,[i for i in range(i,i+2)]).tolist())
            #     i += 2
            elif i + 1 < len(lst) and [l for l in lst[i: i + 2]] == [
                fusing_layers[3],
                fusing_layers[2],
            ]:
                mapped_layers['Linear_ReLU'].append(
                    np.take(name_list, [i for i in range(i, i + 2)]).tolist()
                )
                i += 2
            elif i + 1 < len(lst) and [l for l in lst[i: i + 2]] == [
                fusing_layers[3],
                fusing_layers[4],
            ]:
                mapped_layers['Linear_BatchNorm1d'].append(
                    np.take(name_list, [i for i in range(i, i + 2)]).tolist()
                )
                i += 2
            else:
                i += 1

    detect_sequences(layer_types)

    for keys, value in mapped_layers.items():
        mapped_layers[keys] = np.asarray(mapped_layers[keys])

    mapped_layers['name_type_shape'] = name_type_shape
    # self.mapped_layers = mapped_layers

    # CWP
    keys_to_lookout = ['Conv2d_BatchNorm2d_ReLU', 'Conv2d_BatchNorm2d']
    pruning_layer_of_interest, qat_layer_of_interest = [], []

    # CWP or QAT Fusion Layers
    for keys in keys_to_lookout:
        data = mapped_layers[keys]
        if (len(data) != 0):
            qat_layer_of_interest.append(data)
    mapped_layers['qat_layers'] = np.asarray(qat_layer_of_interest)

    return mapped_layers

def cwp_possible_layers(layer_name_list):
    possible_indices = []
    # for idx in range(len(layer_shape_list)):
    idx = 0
    while idx < len(layer_name_list):
        
        current_value = layer_name_list[idx]
        layer_shape = eval(current_value).weight.shape
        curr_merge_list = []
        curr_merge_list.append([current_value, 0])
        hit_catch = False
        for internal_idx in range(idx + 1, len(layer_name_list) - 1):


            new_layer = layer_name_list[internal_idx]
            new_layer_shape = eval(new_layer).weight.shape

            if (len(new_layer_shape) == 4):
                curr_merge_list.append([new_layer, 0])
            if (layer_shape[0] == new_layer_shape[1]):
                hit_catch = True
                break

            elif (len(new_layer_shape) == 1):
            #                 ipdb.set_trace()
                curr_merge_list[len(curr_merge_list) - 1][1] = new_layer

        possible_indices.append(curr_merge_list)
        if (hit_catch == True):

            idx = internal_idx
        else:
            idx += 1

    return possible_indices

In [9]:

import copy


# mobilenet_v3 = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v3_small', pretrained=True)
# mobilenet_v3 = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

pruned_model  = copy.deepcopy(VGG())


mapped_layers = layer_mapping(pruned_model)
name_list = mapped_layers['name_type_shape'][:, 0]
conv_list = name_list[mapped_layers['name_type_shape'][:, 1]=='Conv2d']
total_convs = len(conv_list)




In [25]:
import ipdb
DG = tp.DependencyGraph().build_dependency(pruned_model, example_inputs=test_input)
ignored_layers = []

for m in pruned_model.modules():
    if isinstance(m, torch.nn.Linear):
        ignored_layers.append(m) # DO NOT prune the final classifier!

ignore_last_conv=True

for group in DG.get_all_groups(ignored_layers=ignored_layers, root_module_types=[nn.Conv2d]):
    if(ignore_last_conv):
        ignore_last_conv=False
        continue
    else:
        
        for i, (dep, idxs) in enumerate(group):
            out_layer = dep.layer
            
            if(isinstance(out_layer, torch.nn.Conv2d)):
                
                prev_conv = out_layer
                root_idxs = group[i].root_idxs
                next_bn, next_convs = [], []
                first_bn_flag = True
                
                for n_lid in range(len(group)):
                    layer = group[n_lid].dep.target.module

                    if isinstance(layer, torch.nn.BatchNorm2d):
                        print(first_bn_flag)
                        if first_bn_flag:
                            prev_bn = layer
                            first_bn_flag = False
                        else:
                            next_bn.append(layer)
                    elif isinstance(layer, torch.nn.Conv2d):
                        next_convs.append(layer)


                print("Prev:",prev_conv,prev_bn)
                print("Next:",next_convs,next_bn)
                print("++++++++++++++++++++++++++")
        print("-----------------------------------------------------")
        
                        
            

NameError: name 'ipbd' is not defined

In [21]:
VGG()

VGG(
  (backbone): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (pool1): MaxPo

In [42]:


made_it=0
all_layers = []
anomalous_layers = []
DG = tp.DependencyGraph().build_dependency(pruned_model, example_inputs=test_input)
for group in DG.get_all_groups(ignored_layers=[], root_module_types=[nn.Conv2d]):
    
    dep = group[0][0]
    layer_name = dep.target._name
    layer= dep.target.module
    new_layer_name = reformat_layer_name(layer_name)
 
   
    if(isinstance(layer,torch.nn.Conv2d)):
       
        
        out_channels = layer.out_channels
        in_channels = layer.in_channels
        groups = layer.groups

        
        if(groups == in_channels):
            type_conv_prune= tp.prune_depthwise_conv_in_channels
            channels_to_prune = in_channels
            
        else:
            type_conv_prune= tp.prune_conv_out_channels
            channels_to_prune = out_channels
            
    
        result_array = generate_random_array(channels_to_prune)
        prune_layer = eval('pruned_model.'+str(new_layer_name))
                           
        internal_group = DG.get_pruning_group( prune_layer, type_conv_prune, idxs=result_array )
        
        if DG.check_pruning_group(internal_group): 
                internal_group.prune()
        
        try:
            _ = pruned_model(test_input)
            DG = tp.DependencyGraph().build_dependency(pruned_model, example_inputs=test_input)
            made_it+=1
            
        except:
            anomalous_layers.append("model."+new_layer_name)
            print("Ignore:",new_layer_name, type_conv_prune)
      
            
            


In [43]:
made_it

8

In [12]:
# conv_list = list(conv_list)
# for i in anomalous_layers:
#     conv_list.remove(i)

In [6]:
import torch
import ipdb
from torch_pruning.dependency import Group
from torch_pruning.pruner.importance import Importance

class FrobeniusNormImportance(Importance):
    def __init__(self, p='fro', group_reduction='mean', normalizer='mean', bias=False):
        self.p = p
        self.group_reduction = group_reduction
        self.normalizer = normalizer
        self.bias = bias
    def get_input_channel_importance(self, weight):
        
        in_channels = weight.shape[1]
        importances = []
        # compute the importance for each input channel
        for i_c in range(weight.shape[1]):
            channel_weight = weight.detach()[:, i_c]

            importance = torch.norm(channel_weight)

            importances.append(importance.view(1))
        return torch.cat(importances)
    
    
    def get_num_channels_to_keep(self, channels: int, prune_ratio: float) -> int:
        """A function to calculate the number of layers to PRESERVE after pruning
        Note that preserve_rate = 1. - prune_ratio
        """

        return int(round(channels * (1. - prune_ratio)))


    def __call__(self, group: Group):
        group_imp = []
        group_idxs = []

        for i, (dep, idxs) in enumerate(group):
            print(group[i])
            
    
#             layer = dep.layer
#             root_idxs = group[i].root_idxs
#             ipdb.set_trace()
#             final_layer = group[-1].dep.target.module
#             out_prune_dim = group[-1].dep.target.pruning_dim

#             if( (isinstance(layer, (torch.nn.Conv2d))) and (isinstance(final_layer, (torch.nn.Conv2d)))):                


                
#                 print(layer, final_layer,out_prune_dim )
               

#                 input_conv_weights = input_conv.weight.clone().detach()
#                 out_conv_weights = output_conv.weight.clone().detach()
#                 out_conv_sort_imp = self.get_input_channel_importance(out_conv_weights)
#                 sort_idx = torch.argsort(out_conv_sort_imp, descending=True) 
#                 sorted_input_conv_weights = torch.index_select(input_conv_weights, 0, sort_idx)

#                 original_channels = input_conv.out_channels  # same as next_conv.in_channels
#                 sorted_n_keep = self.get_num_channels_to_keep(original_channels, 0.30) #[:n_keep]
#                 sorted_picks = sorted_input_conv_weights[:sorted_n_keep] #Might change Check later
#                 local_imp = []
#                 for sorted_pick in sorted_picks:
#                     indx = input_conv_weights.index(sorted_pick)
                    


#                 ipdb.set_trace()
# #                 group_imp.append(local_imp)
# #                 group_idxs.append(root_idxs)


#                 if len(group_imp) == 0:
#                     return None

#                 group_imp = torch.cat(group_imp)
#             else:
#                 return None

#         return group_imp


In [5]:
#Meta Pruning


import torch
from torchvision.models import resnet18
import torch_pruning as tp

model = VGG()
checkpoint = torch.load('./vgg.cifar.pretrained.pth',map_location='cpu')
model.load_state_dict(checkpoint['state_dict'])

example_inputs = torch.randn(1, 3, 32, 32)


# 1. Importance criterion
# imp  = tp.importance.MagnitudeImportance(p=2, normalizer=None, target_types=[torch.nn.Conv2d])
# imp  = tp.importance.GroupNormImportance(p=2,normalizer=None,group_reduction="first")#normalizer=None, target_types=[torch.nn.Conv2d])
imp  = FrobeniusNormImportance()
# imp = tp.importance.MagnitudeImportance(p=2,normalizer=None, target_types=[torch.nn.Conv2d])
# 2. Initialize a pruner with the model and the importance criterion
ignored_layers = []
for m in model.modules():
    if isinstance(m, torch.nn.Linear) :
        ignored_layers.append(m) # DO NOT prune the final classifier!
        
pruning_ratio_dict= { model.backbone.conv0 : 0.40000000000000013, model.backbone.conv1 : 0.15000000000000002, model.backbone.conv2: 0.1, model.backbone.conv3: 0.15000000000000002, model.backbone.conv4 : 0.1, model.backbone.conv5 : 0.1, model.backbone.conv6 : 0.20000000000000004} 

pruner = tp.pruner.MetaPruner( # We can always choose MetaPruner if sparse training is not required.
    model,
    example_inputs,
    importance=imp,
    pruning_ratio_dict = pruning_ratio_dict,
    ignored_layers=ignored_layers,
)

pruner.step()
model.zero_grad()

# finetune the pruned model here
# finetune(model)
# ...

> /var/folders/cr/np70dxds62s1rhj2mtr8hlw80000gn/T/ipykernel_17961/3506461137.py(44)__call__()
     43             ipdb.set_trace()
---> 44             final_layer = group[-1].dep.target.module
     45             out_prune_dim = group[-1].dep.target.pruning_dim

ipdb> exit


In [26]:
image_size = 32
transforms = {
    "train": Compose([
        RandomCrop(image_size, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
    ]),
    "test": ToTensor(),
}
dataset = {}
for split in ["train", "test"]:
  dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transforms[split],
  )
dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=512,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
  )



from sconce import sconce

sconces = sconce()
sconces.model= model # Model Definition
sconces.criterion = nn.CrossEntropyLoss() # Loss
sconces.optimizer= optim.Adam(sconces.model.parameters(), lr=1e-4)
sconces.scheduler = optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)
sconces.dataloader = dataloader
sconces.epochs = 5 #Number of time we iterate over the data
sconces.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sconces.experiment_name = "vgg-gmp" # Define your experiment name here

sconces.evaluate()

Files already downloaded and verified
Files already downloaded and verified


test:   0%|          | 0/20 [00:00<?, ?it/s]

62.965931863727455